<a href="https://colab.research.google.com/github/2303A51618/AI-Assistant-Coding/blob/main/AI_ASST_8_3_V.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import re

def is_valid_email(email):
    """
    Validates an email address based on a set of rules.

    Args:
        email (str): The email address to validate.

    Returns:
        bool: True if the email is valid, False otherwise.
    """

    # Rule 1: Must contain '@' and '.' characters
    if '@' not in email or '.' not in email:
        return False

    # Rule 3: Should not allow multiple '@' symbols
    if email.count('@') != 1:
        return False

    local_part, domain_part = email.split('@')

    # Rule 4: The part before '@' (local part) should not be empty
    if not local_part:
        return False

    # Rule 5: The part after '@' (domain part) should not be empty
    if not domain_part:
        return False

    # Rule 6: The domain part must contain at least one '.' after the '@'
    if '.' not in domain_part:
        return False

    # --- Additional structural checks based on common email validation requirements ---

    # Check for consecutive dots in local part and domain part
    if '..' in local_part:
        return False
    if '..' in domain_part:
        return False

    # Rule 2 (part 1): Local part must not start or end with specific special characters including dot.
    # Allowed local part chars: a-zA-Z0-9!#$%&'*+-/=?^_`{|}~.
    # The pattern checks for any of these at the start or end.
    if re.search(r"^[!#$%&'*+-/=?^_`{|}~.]|[!#$%&'*+-/=?^_`{|}~.]$", local_part):
        return False

    # Check for invalid characters in local part (e.g., spaces, other non-allowed unicode)
    # This regex matches the allowed characters for the local part as per RFC, excluding structural rules.
    local_part_allowed_chars_regex = re.compile(r"^[a-zA-Z0-9!#$%&'*+/=?^_`{|}~.-]+$")
    if not local_part_allowed_chars_regex.match(local_part):
        return False

    # Rule 2 (part 2): Domain part must not start or end with hyphen or dot for the *entire* domain_part.
    if domain_part.startswith('-') or domain_part.endswith('-') or \
       domain_part.startswith('.') or domain_part.endswith('.'):
        return False

    # Further validation for domain labels (parts separated by dots)
    domain_labels = domain_part.split('.')
    for i, label in enumerate(domain_labels):
        if not label: # Disallows empty labels, e.g., 'test@.com' or 'test@example..com' (if not caught by '..' check)
            return False
        # Each domain label must not start or end with a hyphen
        if label.startswith('-') or label.endswith('-'): # Handles test@example-.com
            return False
        # Each domain label must contain alphanumeric characters and hyphens only, starting and ending with alphanumeric.
        # This regex covers general character set and hyphen placement for host labels.
        if not re.fullmatch(r"^[a-zA-Z0-9](?:[a-zA-Z0-9-]{0,61}[a-zA-Z0-9])?$", label):
            return False

    # Rule 7: The top-level domain (TLD) must be at least 2 characters long and only alphabetic
    tld = domain_labels[-1]
    if len(tld) < 2 or not tld.isalpha(): # 'test@example.c' or 'test@example.123'
        return False

    return True

## AI-Generated Test Cases

In [5]:
test_cases = {
    # Valid email formats
    "test@example.com": True,
    "john.doe@sub.domain.co.uk": True,
    "user123@mail-server.net": True,
    "firstName.lastName@company.info": True,
    "email_address@example.org": True,
    "email+tag@example.com": True, # Plus addressing is generally valid
    "email@123.com": True, # Numeric domains are valid
    "a@b.co": True, # Minimal valid

    # Invalid email formats based on rules
    # Rule 1: Must contain '@' and '.' characters
    "testexample.com": False,  # Missing '@'
    "test@examplecom": False,  # Missing '.' in domain
    "testexamplecom": False,   # Missing both

    # Rule 2: Must not start or end with special characters
    "@testexample.com": False, # Starts with '@'
    ".test@example.com": False, # Local part starts with '.'
    "test@.example.com": False, # Domain part starts with '.'
    "test@example.com.": False, # Domain part ends with '.'
    "test@example.co-": False,  # Domain part ends with '-'
    "-test@example.com": False, # Local part starts with '-'
    "test.@example.com": False, # Local part ends with '.' (often disallowed by strict parsers)
    "test@example-.com": False, # Hyphen before dot in domain

    # Rule 3: Should not allow multiple '@' symbols
    "test@@example.com": False,
    "test@example@domain.com": False,

    # Rule 4: The part before '@' (local part) should not be empty
    "@example.com": False,

    # Rule 5: The part after '@' (domain part) should not be empty
    "test@": False,

    # Rule 6: The domain part must contain at least one '.' after the '@'
    "test@example": False,

    # Rule 7: The top-level domain (TLD) must be at least 2 characters long
    "test@example.c": False, # TLD is 1 character
    "test@example.a": False, # TLD is 1 character

    # Other invalid cases covered by the regex or implicit rules
    "test..user@example.com": False, # Consecutive dots in local part
    "invalid user@example.com": False, # Space in local part
    "test\u00A0@example.com": False, # Unicode whitespace
    "test@.com": False, # Domain starts with dot
    "test@-example.com": False, # Domain starts with hyphen
    "test@example..com": False # Consecutive dots in domain
}

## Running Test Cases

In [6]:
print("Running email validation test cases...")
print("---------------------------------------")

all_tests_passed = True
passed_count = 0
failed_count = 0

for email, expected_result in test_cases.items():
    actual_result = is_valid_email(email)
    if actual_result == expected_result:
        print(f"Email: '{email}' -> PASSED (Expected: {expected_result}, Got: {actual_result})")
        passed_count += 1
    else:
        print(f"Email: '{email}' -> FAILED (Expected: {expected_result}, Got: {actual_result})")
        all_tests_passed = False
        failed_count += 1

print("---------------------------------------")
if all_tests_passed:
    print(f"All {passed_count} test cases passed successfully!")
else:
    print(f"{passed_count} tests passed, {failed_count} tests failed. Please review the failed cases.")

Running email validation test cases...
---------------------------------------
Email: 'test@example.com' -> PASSED (Expected: True, Got: True)
Email: 'john.doe@sub.domain.co.uk' -> PASSED (Expected: True, Got: True)
Email: 'user123@mail-server.net' -> PASSED (Expected: True, Got: True)
Email: 'firstName.lastName@company.info' -> PASSED (Expected: True, Got: True)
Email: 'email_address@example.org' -> PASSED (Expected: True, Got: True)
Email: 'email+tag@example.com' -> PASSED (Expected: True, Got: True)
Email: 'email@123.com' -> PASSED (Expected: True, Got: True)
Email: 'a@b.co' -> PASSED (Expected: True, Got: True)
Email: 'testexample.com' -> PASSED (Expected: False, Got: False)
Email: 'test@examplecom' -> PASSED (Expected: False, Got: False)
Email: 'testexamplecom' -> PASSED (Expected: False, Got: False)
Email: '@testexample.com' -> PASSED (Expected: False, Got: False)
Email: '.test@example.com' -> PASSED (Expected: False, Got: False)
Email: 'test@.example.com' -> PASSED (Expected: Fa

In [7]:
def assign_grade(score):
    """
    Assigns a letter grade based on the given score.

    Args:
        score (int or float): The numerical score.

    Returns:
        str: The letter grade ('A', 'B', 'C', 'D', 'F') or 'Invalid Input' for invalid scores.
    """
    # Handle invalid input types
    if not isinstance(score, (int, float)):
        return "Invalid Input"

    # Handle scores outside the valid range [0, 100]
    if not (0 <= score <= 100):
        return "Invalid Input"

    # Assign grades based on the specified scale
    if 90 <= score <= 100:
        return 'A'
    elif 80 <= score <= 89:
        return 'B'
    elif 70 <= score <= 79:
        return 'C'
    elif 60 <= score <= 69:
        return 'D'
    else: # score < 60
        return 'F'

## AI-Generated Test Cases for `assign_grade`

In [10]:
test_cases_grades = {
    # Valid scores covering each grade range
    95: 'A', # Mid A
    90: 'A', # Boundary A
    85: 'B', # Mid B
    89: 'B', # Boundary B
    80: 'B', # Boundary B
    75: 'C', # Mid C
    79: 'C', # Boundary C
    70: 'C', # Boundary C
    65: 'D', # Mid D
    69: 'D', # Boundary D
    60: 'D', # Boundary D
    50: 'F', # Mid F
    0: 'F',  # Boundary F

    # Invalid inputs
    -5: 'Invalid Input',    # Below range
    101: 'Invalid Input',   # Above range
    100.1: 'Invalid Input', # Above range (float)
    -0.1: 'Invalid Input',  # Below range (float)
    "eighty": 'Invalid Input', # Non-numeric string
    'NoneType': 'Invalid Input',     # None type, represented as string key
    'ListInput': 'Invalid Input',       # List type, represented as string key
    'DictInput': 'Invalid Input'        # Dictionary type, represented as string key
}

## Running Test Cases for `assign_grade`

In [11]:
print("Running grade assignment test cases...")
print("---------------------------------------")

all_tests_passed_grades = True
passed_count_grades = 0
failed_count_grades = 0

for score_key, expected_grade in test_cases_grades.items():
    score = score_key # Default to using the key directly for scores
    # Convert string keys representing non-numeric types to their actual types for testing
    if score_key == 'NoneType':
        score = None
    elif score_key == 'ListInput':
        score = []
    elif score_key == 'DictInput':
        score = {}

    actual_grade = assign_grade(score)
    if actual_grade == expected_grade:
        print(f"Score: {repr(score_key)} -> PASSED (Expected: '{expected_grade}', Got: '{actual_grade}')")
        passed_count_grades += 1
    else:
        print(f"Score: {repr(score_key)} -> FAILED (Expected: '{expected_grade}', Got: '{actual_grade}')")
        all_tests_passed_grades = False
        failed_count_grades += 1

print("---------------------------------------")
if all_tests_passed_grades:
    print(f"All {passed_count_grades} test cases passed successfully!")
else:
    print(f"{passed_count_grades} tests passed, {failed_count_grades} tests failed. Please review the failed cases.")

Running grade assignment test cases...
---------------------------------------
Score: 95 -> PASSED (Expected: 'A', Got: 'A')
Score: 90 -> PASSED (Expected: 'A', Got: 'A')
Score: 85 -> PASSED (Expected: 'B', Got: 'B')
Score: 89 -> PASSED (Expected: 'B', Got: 'B')
Score: 80 -> PASSED (Expected: 'B', Got: 'B')
Score: 75 -> PASSED (Expected: 'C', Got: 'C')
Score: 79 -> PASSED (Expected: 'C', Got: 'C')
Score: 70 -> PASSED (Expected: 'C', Got: 'C')
Score: 65 -> PASSED (Expected: 'D', Got: 'D')
Score: 69 -> PASSED (Expected: 'D', Got: 'D')
Score: 60 -> PASSED (Expected: 'D', Got: 'D')
Score: 50 -> PASSED (Expected: 'F', Got: 'F')
Score: 0 -> PASSED (Expected: 'F', Got: 'F')
Score: -5 -> PASSED (Expected: 'Invalid Input', Got: 'Invalid Input')
Score: 101 -> PASSED (Expected: 'Invalid Input', Got: 'Invalid Input')
Score: 100.1 -> PASSED (Expected: 'Invalid Input', Got: 'Invalid Input')
Score: -0.1 -> PASSED (Expected: 'Invalid Input', Got: 'Invalid Input')
Score: 'eighty' -> PASSED (Expected: '

In [12]:
import re

def is_sentence_palindrome(sentence):
    """
    Checks if a sentence is a palindrome, ignoring case, spaces, and punctuation.

    Args:
        sentence (str): The sentence to check.

    Returns:
        bool: True if the sentence is a palindrome, False otherwise.
    """
    # Convert to lowercase and remove non-alphanumeric characters
    cleaned_text = re.sub(r'[^a-z0-9]', '', sentence.lower())

    # Compare the cleaned text with its reverse
    return cleaned_text == cleaned_text[::-1]

## AI-Generated Test Cases for `is_sentence_palindrome`

In [13]:
test_cases_palindrome = {
    # Valid Palindromes
    "A man, a plan, a canal: Panama": True,
    "Racecar": True,
    "No lemon, no melon.": True,
    "Madam": True,
    " nurses run ": True,
    "Was it a car or a cat I saw?": True,
    "Eva, can I stab bats in a cave?": True,
    "": True, # Empty string is a palindrome
    "a": True, # Single character is a palindrome
    "1": True, # Single digit is a palindrome
    "!@#$": True, # Only punctuation, cleaned to empty string

    # Non-Palindromes
    "Hello, World!": False,
    "Python is fun": False,
    "Not a palindrome": False,
    "Abcdefg": False,
    "12345": False,
    "Palindrome Test": False,
    "A man a plan a canal Panama.": True, # Should be True as per example, includes punctuation
    "ab": False
}

## Running Test Cases for `is_sentence_palindrome`

In [14]:
print("Running palindrome test cases...")
print("----------------------------------")

all_tests_passed_palindrome = True
passed_count_palindrome = 0
failed_count_palindrome = 0

for sentence, expected_result in test_cases_palindrome.items():
    actual_result = is_sentence_palindrome(sentence)
    if actual_result == expected_result:
        print(f"Sentence: '{sentence}' -> PASSED (Expected: {expected_result}, Got: {actual_result})")
        passed_count_palindrome += 1
    else:
        print(f"Sentence: '{sentence}' -> FAILED (Expected: {expected_result}, Got: {actual_result})")
        all_tests_passed_palindrome = False
        failed_count_palindrome += 1

print("----------------------------------")
if all_tests_passed_palindrome:
    print(f"All {passed_count_palindrome} test cases passed successfully!")
else:
    print(f"{passed_count_palindrome} tests passed, {failed_count_palindrome} tests failed. Please review the failed cases.")

Running palindrome test cases...
----------------------------------
Sentence: 'A man, a plan, a canal: Panama' -> PASSED (Expected: True, Got: True)
Sentence: 'Racecar' -> PASSED (Expected: True, Got: True)
Sentence: 'No lemon, no melon.' -> PASSED (Expected: True, Got: True)
Sentence: 'Madam' -> PASSED (Expected: True, Got: True)
Sentence: ' nurses run ' -> PASSED (Expected: True, Got: True)
Sentence: 'Was it a car or a cat I saw?' -> PASSED (Expected: True, Got: True)
Sentence: 'Eva, can I stab bats in a cave?' -> PASSED (Expected: True, Got: True)
Sentence: '' -> PASSED (Expected: True, Got: True)
Sentence: 'a' -> PASSED (Expected: True, Got: True)
Sentence: '1' -> PASSED (Expected: True, Got: True)
Sentence: '!@#$' -> PASSED (Expected: True, Got: True)
Sentence: 'Hello, World!' -> PASSED (Expected: False, Got: False)
Sentence: 'Python is fun' -> PASSED (Expected: False, Got: False)
Sentence: 'Not a palindrome' -> PASSED (Expected: False, Got: False)
Sentence: 'Abcdefg' -> PASSED (E

In [18]:
from datetime import datetime

def convert_date_format(date_str):
    """
    Converts a date string from 'YYYY-MM-DD' format to 'DD-MM-YYYY' format.

    Args:
        date_str (str): The input date string in 'YYYY-MM-DD' format.

    Returns:
        str: The converted date string in 'DD-MM-YYYY' format, or an error message if the input is invalid.
    """
    if not isinstance(date_str, str):
        return "Invalid Input: Input must be a string."

    try:
        # Parse the input date string from 'YYYY-MM-DD'
        date_object = datetime.strptime(date_str, '%Y-%m-%d')
        # Format the date object to 'DD-MM-YYYY'
        return date_object.strftime('%d-%m-%Y')
    except ValueError:
        return "Invalid Date Format: Expected YYYY-MM-DD."

## AI-Generated Test Cases for `convert_date_format`

In [21]:
test_cases_date_conversion = {
    # Valid conversions
    "2023-10-15": "15-10-2023",
    "1999-01-01": "01-01-1999",
    "2000-02-29": "29-02-2000", # Leap year
    "2024-12-31": "31-12-2024",
    "2005-07-20": "20-07-2005",

    # Invalid input formats
    "10-15-2023": "Invalid Date Format: Expected YYYY-MM-DD.", # Wrong order
    "2023/10/15": "Invalid Date Format: Expected YYYY-MM-DD.", # Wrong separator
    "2023-13-01": "Invalid Date Format: Expected YYYY-MM-DD.", # Invalid month
    "2023-10-32": "Invalid Date Format: Expected YYYY-MM-DD.", # Invalid day
    "invalid-date": "Invalid Date Format: Expected YYYY-MM-DD.", # Non-date string
    "2023-1-1": "Invalid Date Format: Expected YYYY-MM-DD.",   # Single digit month/day
    "23-10-15": "Invalid Date Format: Expected YYYY-MM-DD.",   # Two digit year

    # Edge cases / Invalid types
    "": "Invalid Date Format: Expected YYYY-MM-DD.", # Empty string
    'NoneTypeInput': "Invalid Input: Input must be a string.", # None type, represented as string key
    'IntTypeInput': "Invalid Input: Input must be a string.", # Integer type, represented as string key
    'ListTypeInput': "Invalid Input: Input must be a string." # List type, represented as string key
}

## Running Test Cases for `convert_date_format`

In [22]:
print("Running date conversion test cases...")
print("---------------------------------------")

all_tests_passed_date_conversion = True
passed_count_date_conversion = 0
failed_count_date_conversion = 0

for input_key, expected_output in test_cases_date_conversion.items():
    # Determine the actual input to pass to the function based on the key
    actual_input = input_key
    if input_key == 'NoneTypeInput':
        actual_input = None
    elif input_key == 'IntTypeInput':
        actual_input = 12345
    elif input_key == 'ListTypeInput':
        actual_input = ["2023-10-15"]

    actual_output = convert_date_format(actual_input)
    if actual_output == expected_output:
        print(f"Input: '{repr(input_key)}' -> PASSED (Expected: '{expected_output}', Got: '{actual_output}')")
        passed_count_date_conversion += 1
    else:
        print(f"Input: '{repr(input_key)}' -> FAILED (Expected: '{expected_output}', Got: '{actual_output}')")
        all_tests_passed_date_conversion = False
        failed_count_date_conversion += 1

print("---------------------------------------")
if all_tests_passed_date_conversion:
    print(f"All {passed_count_date_conversion} test cases passed successfully!")
else:
    print(f"{passed_count_date_conversion} tests passed, {failed_count_date_conversion} tests failed. Please review the failed cases.")

In [15]:
class ShoppingCart:
    """
    A simple ShoppingCart class to manage items, add/remove them, and calculate the total cost.
    """
    def __init__(self):
        self.items = {}

    def add_item(self, name, price):
        """
        Adds an item to the shopping cart.

        Args:
            name (str): The name of the item.
            price (float): The price of the item. Must be a positive number.
        """
        if not isinstance(name, str) or not name:
            print("Error: Item name must be a non-empty string.")
            return
        if not isinstance(price, (int, float)) or price <= 0:
            print("Error: Item price must be a positive number.")
            return

        if name in self.items:
            self.items[name]['quantity'] += 1
        else:
            self.items[name] = {'price': price, 'quantity': 1}
        print(f"Added {name} (price: {price}) to the cart.")

    def remove_item(self, name):
        """
        Removes an item from the shopping cart.
        If multiple quantities exist, one quantity is removed.
        If the last quantity is removed, the item is removed from the cart.

        Args:
            name (str): The name of the item to remove.
        """
        if name not in self.items:
            print(f"Error: {name} not found in the cart.")
            return

        if self.items[name]['quantity'] > 1:
            self.items[name]['quantity'] -= 1
            print(f"Removed one {name}. Quantity remaining: {self.items[name]['quantity']}.")
        else:
            del self.items[name]
            print(f"Removed all {name} from the cart.")

    def total_cost(self):
        """
        Calculates the total cost of all items in the cart.

        Returns:
            float: The total cost.
        """
        total = 0.0
        for item_name, details in self.items.items():
            total += details['price'] * details['quantity']
        return total

    def get_items(self):
        """
        Returns the current items in the cart.
        """
        return self.items

## AI-Generated Test Cases for `ShoppingCart`

In [16]:
class TestShoppingCart:
    def __init__(self):
        self.passed_tests = 0
        self.failed_tests = 0

    def _assert_equals(self, actual, expected, message):
        if actual == expected:
            self.passed_tests += 1
            print(f"PASS: {message}")
        else:
            self.failed_tests += 1
            print(f"FAIL: {message} - Expected: {expected}, Got: {actual}")

    def run_tests(self):
        print("\n--- Running ShoppingCart Tests ---")
        self.test_add_single_item()
        self.test_add_multiple_items()
        self.test_add_duplicate_items()
        self.test_remove_single_item()
        self.test_remove_non_existent_item()
        self.test_remove_multiple_quantities()
        self.test_total_cost_empty_cart()
        self.test_total_cost_with_items()
        self.test_add_remove_sequence()
        self.test_invalid_add_inputs()
        self.test_empty_string_name()

        print("\n--- Test Summary ---")
        print(f"Total tests: {self.passed_tests + self.failed_tests}")
        print(f"Passed: {self.passed_tests}")
        print(f"Failed: {self.failed_tests}")
        if self.failed_tests == 0:
            print("All ShoppingCart tests passed successfully!")
        else:
            print("Some ShoppingCart tests failed. Please review.")

    def test_add_single_item(self):
        cart = ShoppingCart()
        cart.add_item("Apple", 1.0)
        self._assert_equals(cart.get_items(), {"Apple": {"price": 1.0, "quantity": 1}}, "Add single item")
        self._assert_equals(cart.total_cost(), 1.0, "Total cost after adding single item")

    def test_add_multiple_items(self):
        cart = ShoppingCart()
        cart.add_item("Apple", 1.0)
        cart.add_item("Banana", 0.5)
        expected_items = {"Apple": {"price": 1.0, "quantity": 1}, "Banana": {"price": 0.5, "quantity": 1}}
        self._assert_equals(cart.get_items(), expected_items, "Add multiple distinct items")
        self._assert_equals(cart.total_cost(), 1.5, "Total cost after adding multiple distinct items")

    def test_add_duplicate_items(self):
        cart = ShoppingCart()
        cart.add_item("Apple", 1.0)
        cart.add_item("Apple", 1.0)
        expected_items = {"Apple": {"price": 1.0, "quantity": 2}}
        self._assert_equals(cart.get_items(), expected_items, "Add duplicate item")
        self._assert_equals(cart.total_cost(), 2.0, "Total cost after adding duplicate item")

    def test_remove_single_item(self):
        cart = ShoppingCart()
        cart.add_item("Apple", 1.0)
        cart.remove_item("Apple")
        self._assert_equals(cart.get_items(), {}, "Remove single item from cart")
        self._assert_equals(cart.total_cost(), 0.0, "Total cost after removing last item")

    def test_remove_non_existent_item(self):
        cart = ShoppingCart()
        cart.add_item("Apple", 1.0)
        # Suppress print output for this error message during test, if needed
        import io, sys
        captured_output = io.StringIO()
        sys.stdout = captured_output
        cart.remove_item("Orange")
        sys.stdout = sys.__stdout__ # Reset redirect
        self._assert_equals(cart.get_items(), {"Apple": {"price": 1.0, "quantity": 1}}, "Remove non-existent item should not change cart")
        self._assert_equals(captured_output.getvalue().strip(), "Error: Orange not found in the cart.", "Error message for non-existent item")


    def test_remove_multiple_quantities(self):
        cart = ShoppingCart()
        cart.add_item("Apple", 1.0)
        cart.add_item("Apple", 1.0)
        cart.remove_item("Apple")
        self._assert_equals(cart.get_items(), {"Apple": {"price": 1.0, "quantity": 1}}, "Remove one quantity from multiple")
        self._assert_equals(cart.total_cost(), 1.0, "Total cost after partial removal")

    def test_total_cost_empty_cart(self):
        cart = ShoppingCart()
        self._assert_equals(cart.total_cost(), 0.0, "Total cost of an empty cart")

    def test_total_cost_with_items(self):
        cart = ShoppingCart()
        cart.add_item("Item A", 10.0)
        cart.add_item("Item B", 5.0)
        cart.add_item("Item A", 10.0)
        self._assert_equals(cart.total_cost(), 25.0, "Total cost with various items")

    def test_add_remove_sequence(self):
        cart = ShoppingCart()
        cart.add_item("Milk", 3.0)
        cart.add_item("Bread", 2.0)
        cart.add_item("Milk", 3.0)
        cart.remove_item("Bread")
        cart.add_item("Eggs", 2.5)
        expected_items = {"Milk": {"price": 3.0, "quantity": 2}, "Eggs": {"price": 2.5, "quantity": 1}}
        self._assert_equals(cart.get_items(), expected_items, "Complex add/remove sequence - items")
        self._assert_equals(cart.total_cost(), 8.5, "Complex add/remove sequence - total cost")

    def test_invalid_add_inputs(self):
        cart = ShoppingCart()
        # Suppress print output for error messages
        import io, sys
        captured_output = io.StringIO()
        sys.stdout = captured_output

        cart.add_item("Invalid Price", -5.0)
        cart.add_item("Invalid Price Type", "ten")
        cart.add_item(123, 10.0) # Invalid name type

        sys.stdout = sys.__stdout__ # Reset redirect
        self._assert_equals(cart.get_items(), {}, "Invalid add inputs should not add items")
        error_messages = captured_output.getvalue().strip().split('\n')
        self._assert_equals(len(error_messages), 3, "Correct number of error messages for invalid inputs")
        self._assert_equals(error_messages[0], "Error: Item price must be a positive number.", "Error message for negative price")
        self._assert_equals(error_messages[1], "Error: Item price must be a positive number.", "Error message for non-numeric price")
        self._assert_equals(error_messages[2], "Error: Item name must be a non-empty string.", "Error message for non-string name")

    def test_empty_string_name(self):
        cart = ShoppingCart()
        import io, sys
        captured_output = io.StringIO()
        sys.stdout = captured_output
        cart.add_item("", 10.0)
        sys.stdout = sys.__stdout__
        self._assert_equals(cart.get_items(), {}, "Empty string name should not add item")
        self._assert_equals(captured_output.getvalue().strip(), "Error: Item name must be a non-empty string.", "Error message for empty string name")


In [17]:
# Run all tests
tester = TestShoppingCart()
tester.run_tests()